In [1]:
import numpy as np
p = np.array([0.6, 0.4])
sigma = np.array([[0.0287, 0], [0, 0.0017]])
p_var = p @ sigma @ p
risk_contribs = np.diag(p) @ sigma @ p / (p @ sigma @ p)
risk_contribs

array([0.9743493, 0.0256507])

In [2]:
lamb = 4.0
gamma = 0.3
sigma = np.array([[0.0287, 0], [0, 0.0017]])
mu = gamma * np.sqrt(np.diag(sigma))
 
w_star = 1.0 / lamb * np.matrix(sigma).I @ mu
w_star = np.asarray(w_star).flatten()
 
w_var = w_star @ sigma @ w_star     
risk_contribs = np.diag(w_star) @ sigma @ w_star / w_var
risk_contribs

array([0.5, 0.5])

In [3]:
# Risk parity on 3 asset portfolio

import cvxpy as cp
sigma = np.matrix([[0.0225 , 0.0216 , 0.00075],   
                   [0.0216 , 0.0324 , 0.00045], 
                   [0.00075, 0.00045, 0.0025]])
N = sigma.shape[0]
w = cp.Variable(N)
gamma = 1.0
objective = cp.Minimize(0.5 * cp.quad_form(w, sigma) - gamma * sum(cp.log(w)))
constraints = []    
prob = cp.Problem(objective, constraints)
result = prob.solve()
p = w.value
risk_contribs = np.diag(p) @ sigma @ p / (p @ sigma @ p)
risk_contribs

matrix([[0.33333644, 0.33333653, 0.33332703]])

In [4]:
import yfinance as yf
 
tickers = ['VV', 'VB', 'VEA', 'VWO', 'AGG', 'BNDX', 'EMB', 'DJP']
rets = yf.download(tickers, start='2014-01-01', end='2021-12-31')['Adj Close']
rets = rets.pct_change().dropna(axis=0, how='any')[tickers]
sigma = np.matrix(rets.cov().values) * 252
                  
N = sigma.shape[0]                 
w = cp.Variable(N)               
gamma = np.concatenate((np.ones(4) / 4, np.ones(3) / 3, np.ones(1) / 1))
objective = cp.Minimize(0.5 * cp.quad_form(w, sigma) - (gamma @ cp.log(w)))         
constraints = []          
prob = cp.Problem(objective, constraints)
           
result = prob.solve()               
p = w.value
p_var = p @ sigma @ p               
risk_contribs = np.diag(p) @ sigma @ p / (p @ sigma @ p)        
risk_contribs

[*********************100%%**********************]  8 of 8 completed


matrix([[0.08333302, 0.08333361, 0.08333326, 0.08333317, 0.11111168,
         0.11111111, 0.11111578, 0.33332837]])